## Import library

In [43]:
import pandas as pd
import numpy as np
import csv
import json
import requests
from xml.etree import ElementTree
import glob
import re
import sklearn.linear_model as lm

## Load dataset

In [71]:
# points = pd.read_csv('data/2014_point_spread_footballlocks.csv')
points = pd.read_csv('data/2015_point_spread_footballlocks.csv')
points.head()

,eid,season,week,Home,Away,HomeScore,AwayScore,Day,Time,Favorite,Underdog,Spread,CoverOrNot
0,2015091000,2015,1,NE,PIT,28,21,Thu,8:30,NE,PIT,-7.0,0
1,2015091300,2015,1,CHI,GB,23,31,Sun,1:00,GB,CHI,-6.5,1
2,2015091301,2015,1,STL,SEA,34,31,Sun,1:00,SEA,STL,-3.5,-1
3,2015091302,2015,1,JAC,CAR,9,20,Sun,1:00,CAR,JAC,-3.0,1
4,2015091303,2015,1,WAS,MIA,10,17,Sun,1:00,MIA,WAS,-4.0,1


In [72]:
a = points.Underdog.unique()
b = points.Favorite.unique()
c = np.setdiff1d(np.union1d(a, b), np.intersect1d(a, b))
c

array(['SF'], dtype=object)

In [69]:
# ## TODO: change to complete dataset
# covers = []
# for i, r in points.iterrows():
#     h = r['Home']
#     a = r['Away']
#     f = r['Favorite']
#     u = r['Underdog']
#     hs = r['HomeScore']
#     vs = r['AwayScore']
#     fs = -1
#     us = -1
#     if h == f:
#         fs = hs
#         us = vs
#     else:
#         fs = vs
#         us = hs
#     if (fs == -1) or (us == -1):
#         print "Favorite or Underdog not match"
#         print r
#         break
#     if (fs - us) > -r['Spread']:
#         cover = 1
#     elif (fs - us) == -r['Spread']:
#         cover = 0
#     else:
#         cover = -1
#     covers.append(cover)
# points["CoverOrNot"] = pd.Series(covers)

In [66]:
points.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [47]:
points.to_csv("data/2015_point_spread_footballlocks.csv", index=False)

In [48]:
len(points.Favorite.unique())

31

In [49]:
len(points.Underdog.unique())

32

## Go through season 2014, take REG 1-16 as training set, 17 as testing set

In [82]:
def parseMatches(df, eid, points):
    try:
        df = df.loc[:, ['time', 'desc', 'qtr', 'yrdln', 'posteam', 'note', 'down']].copy()
        df = df.sort_values(by=['qtr', 'time'], ascending=[True, False], axis=0).reset_index()

        # [time, Missing Value] make endtime row as 0, since they are nan
        time_null_idx = df['time'].index[pd.isnull(df['time'].values)]
        df.loc[time_null_idx, 'time'] = '0:00'
        df['time'] = pd.Series([float(a.split(':')[0])*60+float(a.split(':')[1]) for a in df['time'].values])

        if len(df.qtr.unique()) == 4:
            df['lefttime'] = 3600 - 15*60*(df['qtr']) + df['time'] # unit: seconds
        else: # suppose only 1 overtime
            df['lefttime'] = 3600 + 15*60 - df.iloc[-1].time - 15*60*(df['qtr']) + df['time'] # unit: seconds


        # [yrdln, Missing Value] Missing Value: make nan the same as previous one
        df['yrdln'] = df['yrdln'].fillna(method='ffill')

        # Assign Home, Away, HomeScore, AwayScore, CoverOrNot to current match
        df['Home'] = points[points['eid'] == eid].Home.iloc[0]
        df['Away'] = points[points['eid'] == eid].Away.iloc[0]
        df['Favorite'] = points[points['eid'] == eid].Favorite.iloc[0]
        df['Underdog'] = points[points['eid'] == eid].Underdog.iloc[0]
        df['HomeScore'] = 0
        df['AwayScore'] = 0
        df['eid'] = eid
        df['CoverOrNot'] = points[points['eid'] == eid].CoverOrNot.iloc[0]
        
        h_inter_i = []
        v_inter_i = []
        h_inter = 0
        v_inter = 0
        
        

        for i, r in df.iterrows():
            # Add score according FG and TD
            if r['note'] == 'TD':
                # In case the touch down has been recovered
                if 'RECOVERED' in r['desc']:
                    rec_team = r['desc'].split("RECOVERED by ", 1)[1][0:3]
                    if not rec_team[-1].isalpha():
                        rec_team = rec_team[0:2]
                    if 'REVERSED' in r['desc'].split("RECOVERED by ", 1)[1]:
                        if r['Home'] == rec_team:
                            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
                            v_inter += 1
                            v_inter_i.append(i)
                        elif r['Away'] == rec_team:
                            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
                            h_inter += 1
                            h_inter_i.append(i)

                    else:
                        if r['Home'] == rec_team:
                            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
                            h_inter += 1
                            h_inter_i.append(i)
                        elif r['Away'] == rec_team:
                            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
                            v_inter += 1
                            v_inter_i.append(i)


                # In case the touch down is catch by kick and run
                elif 'kicks' in r['desc'] and r['posteam'] == r['Home']:
                    df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
                    v_inter += 1
                    v_inter_i.append(i)
                elif 'kicks' in r['desc'] and r['posteam'] == r['Away']:
                    df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
                    h_inter += 1
                    h_inter_i.append(i)




                # In case the touch down complished by intercept
                elif 'INTERCEPTED' in r['desc'] and r['posteam'] == r['Home']:
                    df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
                    v_inter += 1
                    v_inter_i.append(i)
                elif 'INTERCEPTED' in r['desc'] and r['posteam'] == r['Away']: # In case the touch down has been recovered
                    df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
                    h_inter += 1
                    h_inter_i.append(i)

                # In case the touch down complished by punts
                elif 'punts' in r['desc']:
                    if r['Home'] == r['posteam']:
                        df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
                        v_inter += 1
                        v_inter_i.append(i)
                    elif r['Away'] == r['posteam']:
                        df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
                        h_inter += 1
                        h_inter_i.append(i)

                # TODO: check, In case there's other penalty while touch down
#                 elif 'PENALTY' in r['desc']:
#                     pen_team = r['desc'].split("PENALTY on ", 1)[1][0:3]
#                     if r['Home'] == pen_team:
#                         df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
#                     elif r['Away'] == pen_team:
#                         df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6

                # default touchdown
                elif r['posteam'] == r['Home']:
                    df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
                elif r['posteam'] == r['Away']:
                    df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6

            if r['note'] == 'PENALTY':
                if 'TWO-POINT CONVERSION ATTEMPT' in r['desc']:
                    pen_team = r['desc'].split("PENALTY on ", 1)[1][0:3]
                    if not pen_team[-1].isalpha():
                        pen_team = pen_team[0:2]

                    if 'ATTEMPT FAILS' in r['desc']:
                        pass
                    elif 'ATTEMPT SUCCEEDS' in r['desc']:
                        if r['Away'] == pen_team:
                            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 2
                        elif r['Home'] == pen_team:
                            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 2


            if r['note'] == 'FG':
                if r['posteam'] == r['Home']:
                    df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 3
                elif r['posteam'] == r['Away']:
                    df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 3
            if r['note'] == 'XP':
                if r['posteam'] == r['Home']:
                    df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 1
                elif r['posteam'] == r['Away']:
                    df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 1
            if r['note'] == '2PS':
                if r['posteam'] == r['Home']:
                    df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 2
                elif r['posteam'] == r['Away']:
                    df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 2

            if r['note'] == '2PR':
                if r['posteam'] == r['Home']:
                    df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 2
                elif r['posteam'] == r['Away']:
                    df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 2


            if r['note'] == 'SAF':
                if r['posteam'] == r['Home']:
                    df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 2
                elif r['posteam'] == r['Away']:
                    df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 2
            
            # Missing data: 2014112307 2nd quarter starter -- TOUCHDOWN
            if eid == 2014112307 and i == 52:
                df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6

            # yrdln: home 0, away 100
            yrdln_split = r['yrdln'].split(' ')
            if len(yrdln_split) == 1 and yrdln_split[0] == str(50):
                f_yrdln = 50
                u_yrdln = 50
            elif len(yrdln_split) == 2:
                side = r['yrdln'].split(' ')[0]
                yrdln = int(r['yrdln'].split(' ')[1])
                if side == r['Favorite']:
                    f_yrdln = yrdln
                    u_yrdln = 100 - yrdln
                elif side == r['Underdog']:
                    u_yrdln = yrdln
                    f_yrdln = 100 - yrdln
            else:
                print 'Error in parsing yrdln, eid=', eid, 'yrdln=', r['yrdln']

            df.loc[i, 'f_yrdln'] = f_yrdln
            df.loc[i, 'u_yrdln'] = u_yrdln
    except AttributeError:
        print 'AttributeError:', eid, df['time'].values
        
    # Assign Favorite score and Underdog score
    if df['Home'].values[0] == df['Favorite'].values[0]:
        df['FavoriteScore'] = df['HomeScore']
        df['UnderdogScore'] = df['AwayScore']
    elif df['Home'].values[0] == df['Underdog'].values[0]:
        df['FavoriteScore'] = df['AwayScore']
        df['UnderdogScore'] = df['HomeScore']


    ## Verify with final value
    parse_hs = df.iloc[-1]['HomeScore']
    parse_vs = df.iloc[-1]['AwayScore']
    real_hs = points[points['eid'] == eid]['HomeScore'].values[0]
    real_vs = points[points['eid'] == eid]['AwayScore'].values[0]
    
    if parse_hs == real_hs and parse_vs == real_vs:
        print 'Successfully finish:', eid
    else:
        # TODO: check which interception actually had following extra point, right now just start from first one
        if (0 < (real_hs - parse_hs) <= h_inter) or (0 < (real_vs - parse_vs) <= v_inter):
            try:
                for i in range(real_hs - parse_hs):
                    idx = h_inter_i[i]
                    df.loc[idx:, 'HomeScore'] += 1
                    print 'Adjust Home Score', eid
                for i in range(real_vs - parse_vs):
                    idx = v_inter_i[i]
                    df.loc[idx:, 'AwayScore'] += 1
                    print 'Adjust Away Score', eid
            except IndexError:
                print 'IndexError:', h_inter_i, v_inter_i, real_vs - parse_vs

        else:
            print 'Fail finishing:', eid
            print 'parse home score =', df.iloc[-1]['HomeScore'], 'real home score =', points[points['eid'] == eid]['HomeScore'].values[0]
            print 'parse away score =', df.iloc[-1]['AwayScore'], 'real away score =', points[points['eid'] == eid]['AwayScore'].values[0]
    return df

In [83]:
for i, r in points[(points['season'] == 2015) & (points['week'] == 5)].iterrows():
    eid = r['eid']
    plays = pd.read_csv('data/game_data/'+str(eid)+'_plays.csv')
    plays = parseMatches(plays, eid, points)
    f_plays = plays.loc[:, ['qtr', 'lefttime', 'FavoriteScore', 'f_yrdln', 'CoverOrNot']]
    f_plays.rename(columns={
        'qtr': 'qtr',
        'lefttime': 'lefttime',
        'FavoriteScore': 'score',
        'f_yrdln': 'yrdln',
        'CoverOrNot': 'cover',
    }, inplace=True)
    f_plays['team'] = plays.Favorite.unique()[0]
    u_plays = plays.loc[:, ['qtr', 'lefttime', 'UnderdogScore', 'u_yrdln', 'CoverOrNot']]
    u_plays.rename(columns={
        'qtr': 'qtr',
        'lefttime': 'lefttime',
        'UnderdogScore': 'score',
        'u_yrdln': 'yrdln',
        'CoverOrNot': 'cover',
    }, inplace=True)
    u_plays['team'] = plays.Underdog.values[0]
    u_plays['cover'] = -u_plays['cover']

Successfully finish: 2015100800
Adjust Home Score 2015101101
Adjust Away Score 2015101102
Successfully finish: 2015101103
Fail finishing: 2015101104
parse home score = 36 real home score = 38
parse away score = 31 real away score = 31


UnboundLocalError: local variable 'f_yrdln' referenced before assignment

### Validation Block

In [78]:
eid = 2015101104
df = pd.read_csv('data/game_data/'+str(eid)+'_plays.csv')
df = df.loc[:, ['time', 'desc', 'qtr', 'yrdln', 'posteam', 'note']].copy()
df = df.sort_values(by=['qtr', 'time'], ascending=[True, False], axis=0).reset_index()

# [time, Missing Value] make endtime row as 0, since they are nan
time_null_idx = df['time'].index[pd.isnull(df['time'].values)]
df.loc[time_null_idx, 'time'] = '0:00'
df['time'] = pd.Series([float(a.split(':')[0])*60+float(a.split(':')[1]) for a in df['time'].values])
if len(df.qtr.unique()) == 4:
    df['lefttime'] = 3600 - 15*60*(df['qtr']) + df['time'] # unit: seconds
else: # suppose only 1 overtime
    df['lefttime'] = 3600 + 15*60 - df.iloc[-1].time - 15*60*(df['qtr']) + df['time'] # unit: seconds

# [yrdln, Missing Value] Missing Value: make nan the same as previous one
df['yrdln'] = df['yrdln'].fillna(method='ffill')

# Assign Home, Away, HomeScore, AwayScore, CoverOrNot to current match
df['Home'] = points[points['eid'] == eid].Home.iloc[0]
df['Away'] = points[points['eid'] == eid].Away.iloc[0]
df['eid'] = eid

df['HomeScore'] = 0
df['AwayScore'] = 0
df['CoverOrNot'] = points[points['eid'] == eid].CoverOrNot.iloc[0]

h_inter_i = []
v_inter_i = []
h_inter = 0
v_inter = 0
for i, r in df.iterrows():
    # Add score according FG and TD
    if r['note'] == 'TD':
        # In case the touch down has been recovered
        if 'RECOVERED' in r['desc']:
            rec_team = r['desc'].split("RECOVERED by ", 1)[1][0:3]
            if not rec_team[-1].isalpha():
                rec_team = rec_team[0:2]
            if r['Home'] == rec_team:
                df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
                h_inter += 1
                h_inter_i.append(i)
            elif r['Away'] == rec_team:
                df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
                v_inter += 1
                v_inter_i.append(i)


         # In case the touch down is catch by kick and run
        elif 'kicks' in r['desc'] and r['posteam'] == r['Home']:
            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
            v_inter += 1
            v_inter_i.append(i)
        elif 'kicks' in r['desc'] and r['posteam'] == r['Away']:
            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
            h_inter += 1
            h_inter_i.append(i)



        # In case the touch down complished by intercept
        elif 'INTERCEPTED' in r['desc'] and r['posteam'] == r['Home']:
            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
            v_inter += 1
            v_inter_i.append(i)
        elif 'INTERCEPTED' in r['desc'] and r['posteam'] == r['Away']: # In case the touch down has been recovered
            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
            h_inter += 1
            h_inter_i.append(i)

        # In case the touch down complished by punts
        elif 'punts' in r['desc']:
            if r['Home'] == r['posteam']:
                df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
                v_inter += 1
                v_inter_i.append(i)
            elif r['Away'] == r['posteam']:
                df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
                h_inter += 1
                h_inter_i.append(i)

        # TODO: check, In case there's other penalty while touch down
#         elif 'PENALTY' in r['desc']:
#             pen_team = r['desc'].split("PENALTY on ", 1)[1][0:3]
#             if r['Home'] == pen_team:
#                 df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
#             elif r['Away'] == pen_team:
#                 df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6

        # default touchdown
        elif r['posteam'] == r['Home']:
            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 6
            print r['posteam'], r['desc']
        elif r['posteam'] == r['Away']:
            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 6
            print r['posteam'], r['desc']

    if r['note'] == 'PENALTY':
        if 'TWO-POINT CONVERSION ATTEMPT' in r['desc']:
            pen_team = r['desc'].split("PENALTY on ", 1)[1][0:3]
            if not pen_team[-1].isalpha():
                pen_team = pen_team[0:2]

            if 'ATTEMPT FAILS' in r['desc']:
                pass
            elif 'ATTEMPT SUCCEEDS' in r['desc']:
                if r['Away'] == pen_team:
                    df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 2
                elif r['Home'] == pen_team:
                    df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 2


    if r['note'] == '2PR':
        if r['posteam'] == r['Home']:
            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 2
        elif r['posteam'] == r['Away']:
            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 2



    if r['note'] == 'FG':
        if r['posteam'] == r['Home']:
            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 3
        elif r['posteam'] == r['Away']:
            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 3
    if r['note'] == 'XP':
        if r['posteam'] == r['Home']:
            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 1
        elif r['posteam'] == r['Away']:
            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 1
    if r['note'] == '2PS':
        if r['posteam'] == r['Home']:
            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 2
        elif r['posteam'] == r['Away']:
            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 2

    if r['note'] == 'SAF':
        if r['posteam'] == r['Home']:
            df.loc[i:, 'AwayScore'] = df.loc[i:, 'AwayScore'] + 2
        elif r['posteam'] == r['Away']:
            df.loc[i:, 'HomeScore'] = df.loc[i:, 'HomeScore'] + 2



    # yrdln: home 0, away 100
    yrdln_split = r['yrdln'].split(' ')
    if len(yrdln_split) == 1 and yrdln_split[0] == str(50):
        h_yrdln = 50
        v_yrdln = 50
    elif len(yrdln_split) == 2:
        side = r['yrdln'].split(' ')[0]
        yrdln = int(r['yrdln'].split(' ')[1])
        if side == r['Home']:
            h_yrdln = yrdln
            v_yrdln = 100 - yrdln
        elif side == r['Away']:
            v_yrdln = yrdln
            h_yrdln = 100 - yrdln
    else:
        print 'Error in parsing yrdln, eid=', eid, 'yrdln=', r['yrdln']


    df.loc[i, 'h_yrdln'] = h_yrdln
    df.loc[i, 'v_yrdln'] = v_yrdln

## Verify with final value
parse_hs = df.iloc[-1]['HomeScore']
parse_vs = df.iloc[-1]['AwayScore']
real_hs = points[points['eid'] == eid]['HomeScore'].values[0]
real_vs = points[points['eid'] == eid]['AwayScore'].values[0]

if parse_hs == real_hs and parse_vs == real_vs:
    print 'Successfully finish:', eid
else:
    # TODO: check which interception actually had following extra point, right now just start from first one
    if (0 < (real_hs - parse_hs) <= h_inter) or (0 < (real_vs - parse_vs) <= v_inter):
        for i in range(real_hs - parse_hs):
            idx = h_inter_i[i]
            df.loc[idx:, 'HomeScore'] += 1
            print 'Adjust Home Score', eid
        for i in range(real_vs - parse_vs):
            idx = v_inter_i[i]
            df.loc[idx:, 'AwayScore'] += 1
            print 'Adjust Away Score', eid
    else:
        print 'Fail finishing:', eid
        print 'parse home score =', df.iloc[-1]['HomeScore'], 'real home score =', points[points['eid'] == eid]['HomeScore'].values[0]
        print 'parse away score =', df.iloc[-1]['AwayScore'], 'real away score =', points[points['eid'] == eid]['AwayScore'].values[0]

JAC (14:29) (Shotgun) B.Bortles pass short middle to A.Hurns for 12 yards, TOUCHDOWN.
TB (7:34) J.Winston pass short middle to D.Martin for 10 yards, TOUCHDOWN [J.Odrick].
TB (2:31) R.Fragel reported in as eligible.  D.Martin right tackle for 1 yard, TOUCHDOWN.
JAC (:40) (Shotgun) B.Bortles pass short middle to A.Robinson for 13 yards, TOUCHDOWN.
JAC (3:08) (Shotgun) B.Bortles pass short left to T.Yeldon for 4 yards, TOUCHDOWN.
TB (8:30) R.Fragel reported in as eligible.  D.Martin up the middle for 1 yard, TOUCHDOWN.
JAC (1:10) (No Huddle, Shotgun) B.Bortles pass short right to A.Robinson for 5 yards, TOUCHDOWN.
Fail finishing: 2015101104
parse home score = 36 real home score = 38
parse away score = 31 real away score = 31


In [81]:
df[df.qtr == 3]

,index,time,desc,qtr,yrdln,posteam,note,lefttime,Home,Away,eid,HomeScore,AwayScore,CoverOrNot,h_yrdln,v_yrdln
82,121,900.0,C.Barth kicks 70 yards from TB 35 to JAC -5. C...,3,TB 35,TB,KICKOFF,1800.0,TB,JAC,2015101104,20,14,1,35.0,65.0
83,126,893.0,(14:53) B.Bortles pass short middle to A.Robin...,3,JAC 37,JAC,NaN,1793.0,TB,JAC,2015101104,20,14,1,63.0,37.0
84,129,859.0,(14:19) C.Grant left end to TB 45 for 11 yards...,3,JAC 44,JAC,NaN,1759.0,TB,JAC,2015101104,20,14,1,56.0,44.0
85,122,820.0,(13:40) B.Bortles pass deep left to A.Hurns to...,3,TB 45,JAC,NaN,1720.0,TB,JAC,2015101104,20,14,1,45.0,55.0
86,120,794.0,(13:14) (Shotgun) B.Bortles pass short left to...,3,TB 45,JAC,NaN,1694.0,TB,JAC,2015101104,20,14,1,45.0,55.0
87,119,758.0,(12:38) T.Yeldon left end pushed ob at TB 32 f...,3,TB 42,JAC,NaN,1658.0,TB,JAC,2015101104,20,14,1,42.0,58.0
88,125,729.0,(12:09) T.Gerhart up the middle to TB 30 for 2...,3,TB 32,JAC,NaN,1629.0,TB,JAC,2015101104,20,14,1,32.0,68.0
89,130,692.0,(11:32) B.Bortles pass incomplete short middle...,3,TB 30,JAC,NaN,1592.0,TB,JAC,2015101104,20,14,1,30.0,70.0
90,128,686.0,(11:26) (Shotgun) B.Bortles pass short right t...,3,TB 30,JAC,NaN,1586.0,TB,JAC,2015101104,20,14,1,30.0,70.0
91,123,661.0,(11:01) (Shotgun) T.Yeldon right guard to TB 1...,3,TB 15,JAC,NaN,1561.0,TB,JAC,2015101104,20,14,1,15.0,85.0


In [31]:
df.iloc[78].desc, df.iloc[78].posteam

('(2:50) M.Grice up the middle to CAR 1 for no gain (K.Short). FUMBLES (K.Short), RECOVERED by CAR-R.Harper at CAR 1. The Replay Official challenged the fumble ruling, and the play was REVERSED. M.Grice up the middle for 1 yard, TOUCHDOWN.',
 'ARI')

In [408]:
eid = 2014112307
df = pd.read_csv('data/game_data/'+str(eid)+'_plays.csv')
df = df.sort_values(by=['qtr', 'time'], ascending=[True, False], axis=0).reset_index()
df[df.sp == 1]

,index,drive,play,down,time,desc,ydstogo,qtr,ydsnet,yrdln,sp,posteam,note
0,73,1,36,0,15:00,R.Succop kicks 72 yards from TEN 35 to PHI -7....,0,1,0,TEN 35,1,TEN,TD
1,72,1,54,0,14:47,"C.Parkey extra point is GOOD, Center-J.Dorenbo...",0,1,0,TEN 2,1,PHI,XP
14,76,3,334,2,10:49,(10:49) (No Huddle) D.Sproles left end for 4 y...,4,1,51,TEN 4,1,PHI,TD
15,80,3,351,0,10:44,"C.Parkey extra point is GOOD, Center-J.Dorenbo...",0,1,51,TEN 2,1,PHI,XP
41,104,7,982,4,03:28,"(3:28) C.Parkey 36 yard field goal is GOOD, Ce...",18,1,46,TEN 18,1,PHI,FG
52,130,10,1268,0,14:51,"R.Succop extra point is GOOD, Center-B.Brinkle...",0,2,30,PHI 2,1,TEN,XP
59,128,11,1467,4,11:33,"(11:33) C.Parkey 26 yard field goal is GOOD, C...",2,2,73,TEN 7,1,PHI,FG
67,146,12,1642,2,08:26,"(8:26) S.Greene up the middle for 2 yards, TOU...",2,2,81,PHI 2,1,TEN,TD
69,148,12,1659,0,08:22,"R.Succop extra point is GOOD, Center-B.Brinkle...",0,2,81,PHI 2,1,TEN,XP
79,141,13,1906,1,04:37,"(4:37) (No Huddle, Shotgun) L.McCoy left guard...",2,2,69,TEN 2,1,PHI,TD


In [373]:
df.iloc[160].desc, df.iloc[160].posteam

('(Pass formation) TWO-POINT CONVERSION ATTEMPT. T.Bridgewater pass to Ch.Johnson is complete. ATTEMPT SUCCEEDS. PENALTY on GB-M.Neal, Roughing the Passer, 15 yards, enforced between downs.',
 'MIN')

In [75]:
range(2)

[0, 1]

### Create clean dataset

In [36]:
dataset = pd.DataFrame()
for w in range(1, 23):
    for i, r in points[(points['season'] == 2014) & (points['week'] == w)].iterrows():
        eid = r['eid']
        plays = pd.read_csv('data/game_data/'+str(eid)+'_plays.csv')
        plays = parseMatches(plays, eid, points)
        f_plays = plays.loc[:, ['qtr', 'lefttime', 'FavoriteScore', 'f_yrdln', 'CoverOrNot']]
        f_plays.rename(columns={
            'qtr': 'qtr',
            'lefttime': 'lefttime',
            'FavoriteScore': 'score',
            'f_yrdln': 'yrdln',
            'CoverOrNot': 'cover',
        }, inplace=True)
        f_plays['eid'] = eid
        f_plays['season'] = 2014
        f_plays['week'] = w
        f_plays['team'] = plays.Favorite.unique()[0]
        print list(f_plays)
        u_plays = plays.loc[:, ['qtr', 'lefttime', 'UnderdogScore', 'u_yrdln', 'CoverOrNot']]
        u_plays.rename(columns={
            'qtr': 'qtr',
            'lefttime': 'lefttime',
            'UnderdogScore': 'score',
            'u_yrdln': 'yrdln',
            'CoverOrNot': 'cover',
        }, inplace=True)
        u_plays['eid'] = eid
        u_plays['season'] = 2014
        u_plays['week'] = w
        u_plays['team'] = plays.Underdog.values[0]
        u_plays['cover'] = -u_plays['cover']
        dataset = pd.concat([dataset, f_plays, u_plays])
dataset.shape

Successfully finish: 2014090400
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014090700
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014090701
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014090702
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Adjust Home Score 2014090703
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014090704
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014090705
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014090706
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Adjust Home Score 2014090707
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team'

Successfully finish: 2014100511
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Adjust Home Score 2014100512
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014100600
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Adjust Home Score 2014100900
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014101201
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014101202
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014101203
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014101204
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014101205
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team'

Adjust Home Score 2014111000
Adjust Home Score 2014111000
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014111300
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014111601
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014111602
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014111604
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014111600
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014111605
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014111606
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014111607
['qtr', 'lefttime', 'score', 'yrdln', 'cover',

Successfully finish: 2014121411
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014121405
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014121412
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014121413
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014121500
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014121800
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014122000
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Adjust Home Score 2014122001
Adjust Away Score 2014122001
['qtr', 'lefttime', 'score', 'yrdln', 'cover', 'eid', 'season', 'week', 'team']
Successfully finish: 2014122103
['qtr', 'lefttime', 'score', 'yrdln', 'cover',

(94914, 9)

In [37]:
dataset.to_csv('data/2014_baseline_cleandata.csv', index=False)